# Classic semantic segmentation network
For this step, you have to train a classic segmentation network (**DeepLabV2** [2]) on the Cityscapes dataset.
- Dataset: **Cityscapes** [5]
- Training epochs: 50
- Training resolution (Cityscapes): 1024x512
- Test resolution (Cityscapes): 1024x512
- Backbone: **R101** (pre-trained on ImageNet) [2]
- Semantic classes: 19
- Metrics: Mean Intersection over Union (**mIoU**) [read this to understand the metrics], **latency**, **FLOPs**, number of **parameters**.


In [1]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 731.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=de29dee81604a5333d81cd7a4d0500dabe61d3b9fa922a1cea29bd26995c1af8
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=a82b0bbbc92214b9610567ebfb5f1e947c078a684d1738168e40af59fe481bed
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


# 0 - Import libraries

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim

import os
import zipfile
import numpy as np
import time
from PIL import Image
from fvcore.nn import FlopCountAnalysis, flop_count_table

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# 1 - Dataset

In [4]:
class CityScapes(Dataset):
    def __init__(self, root_dir, split='train', image_transform=None, label_transform=None):
        super(CityScapes, self).__init__()
        """
        Args:
            root_dir (string): Directory with all the images and annotations.
            split (string): 'train' or 'val'.
            transform (callable, optional): Optional transform to be applied on a sample.
        """

        self.root_dir = root_dir
        self.split = split
        self.image_transform = image_transform
        self.label_transform = label_transform

        # Get the image and label directories
        self.image_dir = os.path.join(root_dir, 'images', split)
        self.label_dir = os.path.join(root_dir, 'gtFine', split)

        # Get a list of all image files
        self.image_files = []
        for city_dir in os.listdir(self.image_dir):
            city_image_dir = os.path.join(self.image_dir, city_dir)
            self.image_files.extend([os.path.join(city_image_dir, f) for f in os.listdir(city_image_dir) if f.endswith('.png')])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]

        # Get the corresponding label image path
        label_name = img_name.replace('images', 'gtFine').replace('_leftImg8bit', '_gtFine_labelTrainIds')

        # Load image and label
        image = Image.open(img_name).convert('RGB')
        label = Image.open(label_name).convert('L')

        #print(np.unique(np.array(label).reshape(np.array(label).shape[0] * np.array(label).shape[1])).size)

        if self.image_transform:
            image = self.image_transform(image)
        if self.label_transform:
            label = self.label_transform(label)

        #print(np.unique(np.array(label).reshape(np.array(label).shape[0] * np.array(label).shape[1])).size)

        label = torch.Tensor(np.array(label))

        return image, label

# 2 - Models

## 2.1 - DeepLabV2

In [5]:
affine_par = True


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(planes, affine=affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False
        padding = dilation
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,
                               padding=padding, bias=False, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(planes, affine=affine_par)
        for i in self.bn2.parameters():
            i.requires_grad = False
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4, affine=affine_par)
        for i in self.bn3.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)

        return out


class ClassifierModule(nn.Module):
    def __init__(self, inplanes, dilation_series, padding_series, num_classes):
        super(ClassifierModule, self).__init__()
        self.conv2d_list = nn.ModuleList()
        for dilation, padding in zip(dilation_series, padding_series):
            self.conv2d_list.append(
                nn.Conv2d(inplanes, num_classes, kernel_size=3, stride=1, padding=padding,
                          dilation=dilation, bias=True))

        for m in self.conv2d_list:
            m.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.conv2d_list[0](x)
        for i in range(len(self.conv2d_list) - 1):
            out += self.conv2d_list[i + 1](x)
        return out


class ResNetMulti(nn.Module):
    def __init__(self, block, layers, num_classes):
        self.inplanes = 64
        super(ResNetMulti, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64, affine=affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True)  # change
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=1, dilation=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1, dilation=4)
        self.layer6 = ClassifierModule(2048, [6, 12, 18, 24], [6, 12, 18, 24], num_classes)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0, 0.01)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1):
        downsample = None
        if (stride != 1
                or self.inplanes != planes * block.expansion
                or dilation == 2
                or dilation == 4):
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, affine=affine_par))
        for i in downsample._modules['1'].parameters():
            i.requires_grad = False
        layers = []
        layers.append(
            block(self.inplanes, planes, stride, dilation=dilation, downsample=downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        _, _, H, W = x.size()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer6(x)

        x = torch.nn.functional.interpolate(x, size=(H, W), mode='bilinear')

        if self.training == True:
            return x, None, None

        return x

    def get_1x_lr_params_no_scale(self):
        """
        This generator returns all the parameters of the net except for
        the last classification layer. Note that for each batchnorm layer,
        requires_grad is set to False in deeplab_resnet.py, therefore this function does not return
        any batchnorm parameter
        """
        b = []

        b.append(self.conv1)
        b.append(self.bn1)
        b.append(self.layer1)
        b.append(self.layer2)
        b.append(self.layer3)
        b.append(self.layer4)

        for i in range(len(b)):
            for j in b[i].modules():
                jj = 0
                for k in j.parameters():
                    jj += 1
                    if k.requires_grad:
                        yield k

    def get_10x_lr_params(self):
        """
        This generator returns all the parameters for the last layer of the net,
        which does the classification of pixel into classes
        """
        b = []
        if self.multi_level:
            b.append(self.layer5.parameters())
        b.append(self.layer6.parameters())

        for j in range(len(b)):
            for i in b[j]:
                yield i

    def optim_parameters(self, lr):
        return [{'params': self.get_1x_lr_params_no_scale(), 'lr': lr},
                {'params': self.get_10x_lr_params(), 'lr': 10 * lr}]


def get_deeplab_v2(num_classes=19, pretrain=True, pretrain_model_path='DeepLab_resnet_pretrained_imagenet.pth'):
    model = ResNetMulti(Bottleneck, [3, 4, 23, 3], num_classes)

    # Pretraining loading
    if pretrain:
        print('Deeplab pretraining loading...')
        saved_state_dict = torch.load(pretrain_model_path)

        new_params = model.state_dict().copy()
        for i in saved_state_dict:
            i_parts = i.split('.')
            new_params['.'.join(i_parts[1:])] = saved_state_dict[i]
        model.load_state_dict(new_params, strict=False)

    return model

# Define the model (DeepLabV2 with ResNet-101 backbone)
pretrain_model_path = '/kaggle/input/model-weight/deeplab_resnet_pretrained_imagenet.pth'
model = get_deeplab_v2(num_classes=19, pretrain=True, pretrain_model_path=pretrain_model_path).cuda()

Deeplab pretraining loading...


## 2.2 - Bisenet

In [6]:
import warnings
warnings.filterwarnings(action='ignore')

In [7]:
class resnet18(torch.nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.features = models.resnet18(pretrained=pretrained)
        self.conv1 = self.features.conv1
        self.bn1 = self.features.bn1
        self.relu = self.features.relu
        self.maxpool1 = self.features.maxpool
        self.layer1 = self.features.layer1
        self.layer2 = self.features.layer2
        self.layer3 = self.features.layer3
        self.layer4 = self.features.layer4

    def forward(self, input):
        x = self.conv1(input)
        x = self.relu(self.bn1(x))
        x = self.maxpool1(x)
        feature1 = self.layer1(x)  # 1 / 4
        feature2 = self.layer2(feature1)  # 1 / 8
        feature3 = self.layer3(feature2)  # 1 / 16
        feature4 = self.layer4(feature3)  # 1 / 32
        # global average pooling to build tail
        tail = torch.mean(feature4, 3, keepdim=True)
        tail = torch.mean(tail, 2, keepdim=True)
        return feature3, feature4, tail


class resnet101(torch.nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.features = models.resnet101(pretrained=pretrained)
        self.conv1 = self.features.conv1
        self.bn1 = self.features.bn1
        self.relu = self.features.relu
        self.maxpool1 = self.features.maxpool
        self.layer1 = self.features.layer1
        self.layer2 = self.features.layer2
        self.layer3 = self.features.layer3
        self.layer4 = self.features.layer4

    def forward(self, input):
        x = self.conv1(input)
        x = self.relu(self.bn1(x))
        x = self.maxpool1(x)
        feature1 = self.layer1(x)  # 1 / 4
        feature2 = self.layer2(feature1)  # 1 / 8
        feature3 = self.layer3(feature2)  # 1 / 16
        feature4 = self.layer4(feature3)  # 1 / 32
        # global average pooling to build tail
        tail = torch.mean(feature4, 3, keepdim=True)
        tail = torch.mean(tail, 2, keepdim=True)
        return feature3, feature4, tail


def build_contextpath(name):
    model = {
        'resnet18': resnet18(pretrained=True),
        'resnet101': resnet101(pretrained=True)
    }
    return model[name]

In [8]:
class ConvBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=2, padding=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, input):
        x = self.conv1(input)
        return self.relu(self.bn(x))


class Spatial_path(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = ConvBlock(in_channels=3, out_channels=64)
        self.convblock2 = ConvBlock(in_channels=64, out_channels=128)
        self.convblock3 = ConvBlock(in_channels=128, out_channels=256)

    def forward(self, input):
        x = self.convblock1(input)
        x = self.convblock2(x)
        x = self.convblock3(x)
        return x


class AttentionRefinementModule(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.sigmoid = nn.Sigmoid()
        self.in_channels = in_channels
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

    def forward(self, input):
        # global average pooling
        x = self.avgpool(input)
        assert self.in_channels == x.size(1), 'in_channels and out_channels should all be {}'.format(x.size(1))
        x = self.conv(x)
        x = self.sigmoid(self.bn(x))
        # x = self.sigmoid(x)
        # channels of input and x should be same
        x = torch.mul(input, x)
        return x


class FeatureFusionModule(torch.nn.Module):
    def __init__(self, num_classes, in_channels):
        super().__init__()
        # self.in_channels = input_1.channels + input_2.channels
        # resnet101 3328 = 256(from spatial path) + 1024(from context path) + 2048(from context path)
        # resnet18  1024 = 256(from spatial path) + 256(from context path) + 512(from context path)
        self.in_channels = in_channels

        self.convblock = ConvBlock(in_channels=self.in_channels, out_channels=num_classes, stride=1)
        self.conv1 = nn.Conv2d(num_classes, num_classes, kernel_size=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(num_classes, num_classes, kernel_size=1)
        self.sigmoid = nn.Sigmoid()
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

    def forward(self, input_1, input_2):
        x = torch.cat((input_1, input_2), dim=1)
        assert self.in_channels == x.size(1), 'in_channels of ConvBlock should be {}'.format(x.size(1))
        feature = self.convblock(x)
        x = self.avgpool(feature)

        x = self.relu(self.conv1(x))
        x = self.sigmoid(self.conv2(x))
        x = torch.mul(feature, x)
        x = torch.add(x, feature)
        return x


class BiSeNet(torch.nn.Module):
    def __init__(self, num_classes, context_path):
        super().__init__()
        # build spatial path
        self.saptial_path = Spatial_path()

        # build context path
        self.context_path = build_contextpath(name=context_path)

        # build attention refinement module  for resnet 101
        if context_path == 'resnet101':
            self.attention_refinement_module1 = AttentionRefinementModule(1024, 1024)
            self.attention_refinement_module2 = AttentionRefinementModule(2048, 2048)
            # supervision block
            self.supervision1 = nn.Conv2d(in_channels=1024, out_channels=num_classes, kernel_size=1)
            self.supervision2 = nn.Conv2d(in_channels=2048, out_channels=num_classes, kernel_size=1)
            # build feature fusion module
            self.feature_fusion_module = FeatureFusionModule(num_classes, 3328)

        elif context_path == 'resnet18':
            # build attention refinement module  for resnet 18
            self.attention_refinement_module1 = AttentionRefinementModule(256, 256)
            self.attention_refinement_module2 = AttentionRefinementModule(512, 512)
            # supervision block
            self.supervision1 = nn.Conv2d(in_channels=256, out_channels=num_classes, kernel_size=1)
            self.supervision2 = nn.Conv2d(in_channels=512, out_channels=num_classes, kernel_size=1)
            # build feature fusion module
            self.feature_fusion_module = FeatureFusionModule(num_classes, 1024)
        else:
            print('Error: unspport context_path network \n')

        # build final convolution
        self.conv = nn.Conv2d(in_channels=num_classes, out_channels=num_classes, kernel_size=1)

        self.init_weight()

        self.mul_lr = []
        self.mul_lr.append(self.saptial_path)
        self.mul_lr.append(self.attention_refinement_module1)
        self.mul_lr.append(self.attention_refinement_module2)
        self.mul_lr.append(self.supervision1)
        self.mul_lr.append(self.supervision2)
        self.mul_lr.append(self.feature_fusion_module)
        self.mul_lr.append(self.conv)

    def init_weight(self):
        for name, m in self.named_modules():
            if 'context_path' not in name:
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                elif isinstance(m, nn.BatchNorm2d):
                    m.eps = 1e-5
                    m.momentum = 0.1
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)

    def forward(self, input):
        # output of spatial path
        sx = self.saptial_path(input)

        # output of context path
        cx1, cx2, tail = self.context_path(input)
        cx1 = self.attention_refinement_module1(cx1)
        cx2 = self.attention_refinement_module2(cx2)
        cx2 = torch.mul(cx2, tail)
        # upsampling
        cx1 = torch.nn.functional.interpolate(cx1, size=sx.size()[-2:], mode='bilinear')
        cx2 = torch.nn.functional.interpolate(cx2, size=sx.size()[-2:], mode='bilinear')
        cx = torch.cat((cx1, cx2), dim=1)

        if self.training == True:
            cx1_sup = self.supervision1(cx1)
            cx2_sup = self.supervision2(cx2)
            cx1_sup = torch.nn.functional.interpolate(cx1_sup, size=input.size()[-2:], mode='bilinear')
            cx2_sup = torch.nn.functional.interpolate(cx2_sup, size=input.size()[-2:], mode='bilinear')

        # output of feature fusion module
        result = self.feature_fusion_module(sx, cx)

        # upsampling
        result = torch.nn.functional.interpolate(result, scale_factor=8, mode='bilinear')
        result = self.conv(result)

        if self.training == True:
            return result, cx1_sup, cx2_sup

        return result

# 3 - WANDB

## 3.1 - Install WANDB

In [9]:
# WANDB
!pip install -q wandb

# import the library
import wandb

In [10]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## 3.2 - PIPELINE

In [11]:
def model_pipeline(config=None):

    # tell wandb to get started
    with wandb.init(config=config):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, val_loader, criterion, optimizer = make(config)
        #print(model)

        # and use them to train the model
        train(model, train_loader, criterion, optimizer, config)

        # and test its final performance
        val(model, val_loader)

    return model

In [12]:
def make(config):
    # Make the data
    train, test = get_data(train=True), get_data(train=False)
    train_loader = make_loader(train, batch_size=config.batch_size,train=True)
    test_loader = make_loader(test, batch_size=config.batch_size,train=False)

    # Make the model (DeepLabV2 with ResNet-101 backbone)
    pretrain_model_path = '/kaggle/input/model-weight/deeplab_resnet_pretrained_imagenet.pth'
    model = get_deeplab_v2(num_classes=19, pretrain=True, pretrain_model_path=pretrain_model_path).cuda()

    # Make the loss and optimizer
    optimizer = optim.SGD(model.parameters(), 
                          lr=config.learning_rate, 
                          momentum=0.9, 
                          weight_decay=5e-4)
    
    criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
    
    return model, train_loader, test_loader, criterion, optimizer

In [13]:
# Define transforms for preprocessing
image_transform = transforms.Compose([
        transforms.Resize((256,512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

label_transform = transforms.Compose([
        transforms.Resize((256,512)),
    ])

root_dir ='/kaggle/input/cityscapes/Cityscapes/Cityspaces'

def get_data(train=True):
    if train == True:
        # train dataset
        dataset = CityScapes(root_dir=root_dir, split='train', image_transform=image_transform, label_transform=label_transform)
    else:
        # test dataset
        dataset = CityScapes(root_dir=root_dir, split='val', image_transform=image_transform, label_transform=label_transform)
    
    return dataset


def make_loader(dataset, batch_size = 8, train=True):
    if train == True:
        # train dataloader
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True,drop_last=True)
    else:
        # test dataloader
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False,drop_last=True)
    
    return dataloader

In [14]:
def train_log(loss, mIOU, epoch):
    #wandb.log({"epoch": epoch, "loss": loss, "mIOU":mIOU})
    wandb.log({"loss": loss, "mIOU":mIOU}, step= epoch)
    print(f"----------------------------------")
    print(f"Loss after {epoch+1} epochs: {loss:.3f}")
    print(f"mIOU after {epoch+1} epochs: {mIOU:.3f}%")

In [15]:
def id_processing(targets):
    targets = targets.cuda()
    
    # Define valid indices
    valid_indices = torch.tensor(list(range(19)) + [255]).to(targets.device)

    # Replace all IDs not in valid_indices with 255
    processed_targets = torch.where(torch.isin(targets, valid_indices), targets, torch.tensor(255, device=targets.device))

    return processed_targets.long()

In [16]:
def fast_hist(a, b, n):
    """
    a and b are predict and mask respectively
    n is the number of classes
    """
    k = (a >= 0) & (a < n) #assign True if the value is in the range between 0 and 18 (class labels)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape((n, n))


def per_class_iou(hist):
    epsilon = 1e-5
    return (np.diag(hist)) / (hist.sum(1) + hist.sum(0) - np.diag(hist) + epsilon)

def mean_iou(num_classes, pred, target):
    mIOU = 0
    for i in range(len(pred)):
        hist = fast_hist(target[i].cpu().numpy(),pred[i].cpu().numpy(), num_classes)
        IOU = per_class_iou(hist)
        mIOU = mIOU + sum(IOU)/num_classes
    return mIOU

In [17]:
def train(model, dataloader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!

    # Run training and track with wandb
    total_batches = len(dataloader) * config.epoch
    total_images = 0  # number of images seen
    
    for epoch in range(config.epoch):
        running_loss = 0.0
        total_mIOU = 0
        total_images = 0
        
        for batch_id, (inputs, targets) in enumerate(dataloader):

            inputs, targets = inputs.cuda(), id_processing(targets).cuda()
        
            outputs = model(inputs)

            loss = criterion(outputs[0], targets)

            # Backprpagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = outputs[0].max(1)

            running_mIOU = mean_iou(outputs[0].size()[1], predicted, targets)
            total_mIOU += running_mIOU.sum().item()
            total_images += len(predicted)
                
        train_loss = running_loss / len(dataloader)
        mIOU = total_mIOU/total_images
        
        train_log(train_loss, mIOU, epoch)

In [18]:
# Validation method
def val(model, dataloader):
    model.eval()
    total_mIOU = 0
    total_images = 0
    latency_list = []
    FPS_list = []
    
    with torch.no_grad():
        for batch_id, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.cuda(), id_processing(targets).cuda()
            
            start = time.time() # Record start time
            outputs = model(inputs)
            end = time.time() # Record end time

            # Calculate latency for this iteration
            latency_i = end - start
            latency_list.append(latency_i)

            # Calculate FPS for this iteration
            FPS_i = 1 / latency_i
            FPS_list.append(FPS_i)

            _, predicted = outputs.max(1)
            
            running_mIOU = mean_iou(outputs.size()[1], predicted, targets)
            total_mIOU += running_mIOU.sum().item()
            total_images += len(predicted)
        
    mIOU = total_mIOU/total_images
    latency = np.sum(latency_list) / len(latency_list)
    test_FPS = np.sum(FPS_list) / len(FPS_list)
    
    # compute flops and #param
    image, _ = next(iter(dataloader))
    height, width = image.shape[2], image.shape[3]
    zero_image = torch.zeros((1, 3, height, width))
    flops = FlopCountAnalysis(model, zero_image.cuda())
    print(flop_count_table(flops))

    print(f'\n\nmIoU: {(mIoU*100):.3f}%, Latency: {latency:.3f}, FPS: {test_FPS:.3f}')

    wandb.log({"mIOU":mIOU,"latency":latency,"FPS":test_FPS})

In [19]:
# config = dict(
#     epochs=5,
#     classes=19,
#     batch_size=8,
#     learning_rate=0.001,
#     momentum=0.9,
#     weight_decay=5e-4,
#     dataset="Cityscapes",
#     architecture="DeepLabV2")

In [20]:
sweep_config= {
    'name': 'my-sweep',
    'metric': {'name': 'loss', 'goal': 'minimize'}, # the goal is maximize the accuracy
    'method': 'random', # test all possible combinations of the hyperparameters
    'parameters': {'epoch': {'values': [5, 10]},        
                   'learning_rate': {'values': [0.1, 0.001, 0.0001]}, # 2 parameters to optimize during the sweep
                   'batch_size': {'values': [2, 4, 8]}},
}

In [21]:
sweep_id = wandb.sweep(sweep_config, project="mldl_step2")

Create sweep with ID: s0l4y5nx
Sweep URL: https://wandb.ai/polito-merelli/mldl_step2/sweeps/s0l4y5nx


In [ ]:
# Build, train and analyze the model with the pipeline
wandb.agent(sweep_id, model_pipeline, count=5)
# model = model_pipeline(config)

wandb: Agent Starting Run: miw8nw2r with config:
wandb: 	batch_size: 8
wandb: 	epoch: 5
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: leonardo-merelli (polito-merelli). Use `wandb login --relogin` to force relogin


Deeplab pretraining loading...
----------------------------------
Loss after 1 epochs: 1.562
mIOU after 1 epochs: 0.128%
----------------------------------
Loss after 2 epochs: 1.026
mIOU after 2 epochs: 0.167%
----------------------------------
Loss after 3 epochs: 0.897
mIOU after 3 epochs: 0.182%
----------------------------------
Loss after 4 epochs: 0.811
mIOU after 4 epochs: 0.191%
----------------------------------
Loss after 5 epochs: 0.750
mIOU after 5 epochs: 0.199%


Traceback (most recent call last):
  File "/tmp/ipykernel_34/528570164.py", line 16, in model_pipeline
    val(model, val_loader)
  File "/tmp/ipykernel_34/3220026950.py", line 40, in val
    print(flop_count_table(flops))
NameError: name 'flop_count_table' is not defined


loss,█▃▂▂▁
mIOU,▁▅▆▇█
loss,0.75024
mIOU,0.19851


Run miw8nw2r errored:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_34/528570164.py", line 16, in model_pipeline
    val(model, val_loader)
  File "/tmp/ipykernel_34/3220026950.py", line 40, in val
    print(flop_count_table(flops))
NameError: name 'flop_count_table' is not defined

wandb: ERROR Run miw8nw2r errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_34/528570164.py", line 16, in model_pipeline
wandb: ERROR     val(model, val_loader)
wandb: ERROR   File "/tmp/ipykernel_34/3220026950.py", line 40, in val
wandb: ERROR     print(flop_count_table(flops))
wandb: ERROR NameError: name 'flop_count_table' is not defined
wandb: ERROR 
wandb: Agent Starting Run: 7fyfzmjk with con

Deeplab pretraining loading...
----------------------------------
Loss after 1 epochs: 0.723
mIOU after 1 epochs: 0.212%
----------------------------------
Loss after 2 epochs: 0.434
mIOU after 2 epochs: 0.255%
----------------------------------
Loss after 3 epochs: 0.383
mIOU after 3 epochs: 0.269%
----------------------------------
Loss after 4 epochs: 0.354
mIOU after 4 epochs: 0.280%
----------------------------------
Loss after 5 epochs: 0.334
mIOU after 5 epochs: 0.287%
----------------------------------
Loss after 6 epochs: 0.319
mIOU after 6 epochs: 0.292%
----------------------------------
Loss after 7 epochs: 0.308
mIOU after 7 epochs: 0.298%
----------------------------------
Loss after 8 epochs: 0.299
mIOU after 8 epochs: 0.302%
----------------------------------
Loss after 9 epochs: 0.290
mIOU after 9 epochs: 0.306%
----------------------------------
Loss after 10 epochs: 0.283
mIOU after 10 epochs: 0.309%


Traceback (most recent call last):
  File "/tmp/ipykernel_34/528570164.py", line 16, in model_pipeline
    val(model, val_loader)
  File "/tmp/ipykernel_34/3220026950.py", line 40, in val
    print(flop_count_table(flops))
NameError: name 'flop_count_table' is not defined


loss,█▃▃▂▂▂▁▁▁▁
mIOU,▁▄▅▆▆▇▇▇██
loss,0.28341
mIOU,0.30921


Run 7fyfzmjk errored:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_34/528570164.py", line 16, in model_pipeline
    val(model, val_loader)
  File "/tmp/ipykernel_34/3220026950.py", line 40, in val
    print(flop_count_table(flops))
NameError: name 'flop_count_table' is not defined

wandb: ERROR Run 7fyfzmjk errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_34/528570164.py", line 16, in model_pipeline
wandb: ERROR     val(model, val_loader)
wandb: ERROR   File "/tmp/ipykernel_34/3220026950.py", line 40, in val
wandb: ERROR     print(flop_count_table(flops))
wandb: ERROR NameError: name 'flop_count_table' is not defined
wandb: ERROR 
wandb: Agent Starting Run: qn3c44hr with con

Deeplab pretraining loading...
----------------------------------
Loss after 1 epochs: 1.134
mIOU after 1 epochs: 0.162%
----------------------------------
Loss after 2 epochs: 0.741
mIOU after 2 epochs: 0.202%
----------------------------------
Loss after 3 epochs: 0.625
mIOU after 3 epochs: 0.217%


In [ ]:
"""
aggiungere in config:
- la scelta del modello (deeplab e bisenet)
- la scelta dell'optimizer
- la scelta della loss function ()

#### VEDERE COME FUNZIONA wandb.sweep
"""